In [16]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pdb

# Ignore Harmless Warnings
import warnings
warnings.filterwarnings("ignore")

In [24]:
class GoogleNet(nn.Module):
    def __init__(self, in_channels = 3, num_classes = 2):
        super(GoogleNet, self).__init__()
        self.conv1 = conv_block(in_channels = in_channels, out_channels = 64, kernel_size = (7,7),
                               stride = (2,2), padding = (3,3))
        self.maxpool1 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.conv2 = conv_block(64, 192,kernel_size = 3, stride = 1, padding = 1)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.inception3a = Inception_block(192,64,96,128,16,32,32)
        self.inception3b = Inception_block(256,128,128,192,32,96,64)
        self.maxpool3 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.inception4a = Inception_block(480,192,96,208,16,48,64)
        self.inception4b = Inception_block(512,160,112,224,24,64,64)
        self.inception4c = Inception_block(512,128,128,256,24,64,64)
        self.inception4d = Inception_block(512,112,144,288,32,64,64)
        self.inception4e = Inception_block(528,256,160,320,32,128,128)
        self.maxpool4 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.inception5a = Inception_block(832, 256,160,320,32,128,128)
        self.inception5b = Inception_block(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(kernel_size = 7, stride = 1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(1024,num_classes)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)
        
        return x
class Inception_block(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
        
        super(Inception_block, self).__init__()
        self.branch1 = conv_block(in_channels, out_1x1, kernel_size = 1)
        self.branch2 = nn.Sequential(conv_block(in_channels, red_3x3, kernel_size = 1),
                                    conv_block(red_3x3, out_3x3,kernel_size = 3, padding = 1))
        self.branch3 = nn.Sequential(conv_block(in_channels, red_5x5, kernel_size = 1),
                                    conv_block(red_5x5, out_5x5, kernel_size = 5, padding = 2))
        self.branch4 = nn.Sequential(nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1),
                                    conv_block(in_channels, out_1x1pool, kernel_size = 1))
    
    def forward(self, x):
        # Nx filtersx 28 x 28
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)

class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs) # kernel_size = (1,1), (2,2), (3,3)
        self.batchnorm = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))

torch.Size([3, 2])


In [23]:
# Data Augmentation for the training dataset
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(), # Default probability of 0.5
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # Mean and standard deviation for each colour channel
                        [0.229, 0.224, 0.225])
])

# Data Augmentation for the testing dataset
# We dont need to perform random flips or rotations in the dataset
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

root = 'C:\\Users\\shant\\CNN_Architectures_ImageNet_Pytorch\\Dataset\\CATS_DOGS\\'

train_data = datasets.ImageFolder(os.path.join(root, 'train'), transform = train_transform)

test_data = datasets.ImageFolder(os.path.join(root, 'test'), transform = test_transform)

# Set the manual seed
torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size = 10, shuffle=True)
test_loader = DataLoader(test_data, batch_size= 10)

torch.Size([3, 2])


In [ ]:
device  = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Object for the GoogleNet class
Googlenet = GoogleNet(in_channels = 3, num_classes = 2)
Googlenet.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(Googlenet.parameters(), lr = 0.001)

import time

start_time = time.time()

epochs = 3

max_train_batch = 800 # Batch of 10 images --> 8000 Images
max_test_batch = 300 # Batch of 10 images --> 300 Images

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):

  trn_corr = 0
  tst_corr = 0

  for b, (x_train, y_train) in enumerate(train_loader):

    x_train = x_train.cuda()
    y_train = y_train.cuda()

    # Limit the number of batches
    if b == max_train_batch:
      break
    b += 1
        
    y_pred = Googlenet(x_train)
    loss = criterion(y_pred,y_train)
        
    # Tally number of correct predictions
    #predicted = torch.max(y_pred.data,1)[1]
    #batch_corr = (predicted == y_train).sum()
    #trn_corr += batch_corr
        
    # Update the Parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    if b%200 == 0:

      print(f'Epoch {i} LOSS {loss.item()}')
    
    #train_losses.append(loss)
    #train_correct.append(trn_corr)
    
    #TEST SET  
total_time = time.time() - start_time
print(f'Total Time: {total_time/60} minutes')

Epoch 0 LOSS 0.7474378347396851
